In [1]:
# =============================================================================
# FULL SCRIPT (UPDATED FIX): MobileNetV3-Large backbone + Mixup DG
# Fixes the runtime error by using the correct MobileNetV3 feature dim (960).
# =============================================================================

import os
import random
import numpy as np
import pandas as pd
from typing import Dict
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, ConcatDataset

import torchvision.transforms as transforms
import torchvision.models as models

from sklearn.metrics import roc_auc_score, f1_score, roc_curve, average_precision_score
import matplotlib.pyplot as plt
from tqdm import tqdm

# =============================================================================
# (OPTIONAL) HARMONIZATION
# =============================================================================

RUN_HARMONIZATION = True  # set False if you already generated harmonized CSVs


def harmonize_odir(split: str) -> pd.DataFrame:
    base_path = "/kaggle/input/odir-clr/ODIR_CLR"

    if split == "train":
        img_dir = f"{base_path}/Training_ Set/train_images"
        label_file = f"{base_path}/Training_ Set/train_annotation.xlsx"
    elif split == "val":
        img_dir = f"{base_path}/Validation_set/val_images"
        label_file = f"{base_path}/Validation_set/val_annotation.xlsx"
    else:
        img_dir = f"{base_path}/Test_Set/test_images"
        label_file = f"{base_path}/Test_Set/test_annotation.xlsx"

    df = pd.read_excel(label_file)
    label_cols = ['N', 'D', 'G', 'C', 'A', 'H', 'M', 'O']
    harmonized_rows = []

    for _, row in df.iterrows():
        patient_id = row['ID']
        labels = {col: int(row[col]) for col in label_cols}

        if pd.notna(row['Left-Fundus']) and str(row['Left-Fundus']).strip():
            left_path = os.path.join(img_dir, f"{patient_id}_left.jpg")
            harmonized_rows.append({
                'image_path': left_path,
                'dataset': 'ODIR',
                'split': split,
                'ID': patient_id,
                **labels
            })

        if pd.notna(row['Right-Fundus']) and str(row['Right-Fundus']).strip():
            right_path = os.path.join(img_dir, f"{patient_id}_right.jpg")
            harmonized_rows.append({
                'image_path': right_path,
                'dataset': 'ODIR',
                'split': split,
                'ID': patient_id,
                **labels
            })

    return pd.DataFrame(harmonized_rows)


def harmonize_rfmid_v1(split: str) -> pd.DataFrame:
    base_path = "/kaggle/input/retinal-disease-classification"

    if split == "train":
        img_dir = f"{base_path}/Training_Set/Training_Set/Training"
        label_file = f"{base_path}/Training_Set/Training_Set/RFMiD_Training_Labels.csv"
    elif split == "val":
        img_dir = f"{base_path}/Evaluation_Set/Evaluation_Set/Validation"
        label_file = f"{base_path}/Evaluation_Set/Evaluation_Set/RFMiD_Validation_Labels.csv"
    else:
        img_dir = f"{base_path}/Test_Set/Test_Set/Test"
        label_file = f"{base_path}/Test_Set/Test_Set/RFMiD_Testing_Labels.csv"

    df = pd.read_csv(label_file)

    all_disease_cols = ['DR', 'ARMD', 'MH', 'DN', 'MYA', 'BRVO', 'TSLN',
                        'ERM', 'LS', 'MS', 'CSR', 'ODC', 'CRVO', 'TV', 'AH',
                        'ODP', 'ODE', 'ST', 'AION', 'PT', 'RT', 'RS', 'CRS',
                        'EDN', 'RPEC', 'MHL', 'RP', 'CWS', 'CB', 'ODPM',
                        'PRH', 'MNF', 'HR', 'CRAO', 'TD', 'CME', 'PTCR', 'CF',
                        'VH', 'MCA', 'VS', 'BRAO', 'PLQ', 'HPED', 'CL']

    harmonized_rows = []

    for _, row in df.iterrows():
        image_id = row['ID']
        image_path = None
        for ext in ['.png', '.jpg', '.jpeg']:
            candidate = os.path.join(img_dir, f"{image_id}{ext}")
            if os.path.exists(candidate):
                image_path = candidate
                break
        if image_path is None:
            image_path = os.path.join(img_dir, str(image_id))

        # Mapping to 8-label schema: N,D,G,C,A,H,M,O
        N = 1 if row['Disease_Risk'] == 0 else 0
        D = 1 if row.get('DR', 0) == 1 else 0
        G = 1 if row.get('ODC', 0) == 1 else 0
        C = 1 if row.get('MH', 0) == 1 else 0
        A = 1 if row.get('ARMD', 0) == 1 else 0
        H = 1 if row.get('HR', 0) == 1 else 0
        M = 1 if row.get('MYA', 0) == 1 else 0

        used_for_mapping = ['DR', 'ODC', 'MH', 'ARMD', 'HR', 'MYA']
        other_cols = [col for col in all_disease_cols if col not in used_for_mapping]
        O = 1 if any(row.get(col, 0) == 1 for col in other_cols) else 0

        harmonized_rows.append({
            'image_path': image_path,
            'dataset': 'RFMiD_v1',
            'split': split,
            'ID': image_id,
            'N': N, 'D': D, 'G': G, 'C': C, 'A': A, 'H': H, 'M': M, 'O': O
        })

    return pd.DataFrame(harmonized_rows)


def harmonize_rfmid_v2(split: str) -> pd.DataFrame:
    base_path = "/kaggle/input/rdc-version-2/RFDiM2_0"

    if split == "train":
        img_dir = f"{base_path}/Training_set_2/Train_2"
        label_file = f"{base_path}/Training_set_2/RFMiD_2_Training_labels.csv"
    elif split == "val":
        img_dir = f"{base_path}/Validation_set_2/Validation_2"
        label_file = f"{base_path}/Validation_set_2/RFMiD_2_Validation_labels.csv"
    else:
        img_dir = f"{base_path}/Test_set_2/Test_2"
        label_file = f"{base_path}/Test_set_2/RFMiD_2_Testing_labels.csv"

    try:
        df = pd.read_csv(label_file, encoding='utf-8')
    except UnicodeDecodeError:
        df = pd.read_csv(label_file, encoding='latin1')

    df.columns = df.columns.str.strip()

    potential_disease_cols = ['AH', 'AION', 'ARMD', 'BRVO', 'CB', 'CF', 'CL', 'CME',
                              'CNV', 'CRAO', 'CRS', 'CRVO', 'CSR', 'CWS', 'CSC', 'DN',
                              'DR', 'EDN', 'ERM', 'GRT', 'HPED', 'HR', 'LS', 'MCA',
                              'ME', 'MH', 'MHL', 'MS', 'MYA', 'ODC', 'ODE', 'ODP',
                              'ON', 'OPDM', 'PRH', 'RD', 'RHL', 'RTR', 'RP', 'RPEC',
                              'RS', 'RT', 'SOFE', 'ST', 'TD', 'TSLN', 'TV', 'VS',
                              'HTN', 'IIH']

    all_disease_cols = [col for col in potential_disease_cols if col in df.columns]
    harmonized_rows = []

    skipped_count = 0
    found_count = 0

    for _, row in df.iterrows():
        image_id = int(row['ID'])

        image_path = None
        for ext in ['.jpg', '.JPG', '.png', '.PNG', '.jpeg', '.JPEG']:
            candidate = os.path.join(img_dir, f"{image_id}{ext}")
            if os.path.exists(candidate):
                image_path = candidate
                found_count += 1
                break

        if image_path is None:
            skipped_count += 1
            continue

        # v2 uses WNL for Normal
        wnl = row.get('WNL', 0)
        N = 1 if wnl == 1 else 0
        D = 1 if row.get('DR', 0) == 1 else 0
        G = 1 if row.get('ODC', 0) == 1 else 0
        C = 1 if row.get('MH', 0) == 1 else 0
        A = 1 if row.get('ARMD', 0) == 1 else 0

        # H: HTN preferred, else HR fallback
        H = 1 if row.get('HTN', 0) == 1 else 0
        if H == 0 and 'HR' in df.columns:
            H = 1 if row.get('HR', 0) == 1 else 0

        M = 1 if row.get('MYA', 0) == 1 else 0

        used_for_mapping = ['DR', 'ODC', 'MH', 'ARMD', 'HTN', 'HR', 'MYA', 'WNL']
        other_cols = [col for col in all_disease_cols if col not in used_for_mapping]
        O = 1 if any(row.get(col, 0) == 1 for col in other_cols) else 0

        harmonized_rows.append({
            'image_path': image_path,
            'dataset': 'RFMiD_v2',
            'split': split,
            'ID': image_id,
            'N': N, 'D': D, 'G': G, 'C': C, 'A': A, 'H': H, 'M': M, 'O': O
        })

    if skipped_count > 0:
        print(f"RFMiD_v2 {split}: Found {found_count}, Skipped {skipped_count}")

    return pd.DataFrame(harmonized_rows)


def harmonize_all_datasets() -> Dict[str, pd.DataFrame]:
    results = {}
    print("\n" + "="*60)
    print("HARMONIZING DATASETS")
    print("="*60)

    for split in ['train', 'val', 'test']:
        print(f"\nProcessing {split} split...")
        results[f'ODIR_{split}'] = harmonize_odir(split)
        results[f'RFMiD_v1_{split}'] = harmonize_rfmid_v1(split)
        results[f'RFMiD_v2_{split}'] = harmonize_rfmid_v2(split)

    return results


def print_statistics(harmonized_data: Dict[str, pd.DataFrame]):
    label_cols = ['N', 'D', 'G', 'C', 'A', 'H', 'M', 'O']

    print("\n" + "="*60)
    print("DATASET STATISTICS")
    print("="*60)

    print("\n📊 Sample Counts:")
    print("-" * 40)
    for key, df in sorted(harmonized_data.items()):
        print(f"{key:20s}: {len(df):5d} images")

    total_train = sum(len(df) for k, df in harmonized_data.items() if 'train' in k)
    total_val = sum(len(df) for k, df in harmonized_data.items() if 'val' in k)
    total_test = sum(len(df) for k, df in harmonized_data.items() if 'test' in k)

    print("-" * 40)
    print(f"{'Total Train':20s}: {total_train:5d} images")
    print(f"{'Total Val':20s}: {total_val:5d} images")
    print(f"{'Total Test':20s}: {total_test:5d} images")
    print(f"{'Grand Total':20s}: {total_train + total_val + total_test:5d} images")

    for dataset_name in ['ODIR', 'RFMiD_v1', 'RFMiD_v2']:
        dataset_dfs = {k: v for k, v in harmonized_data.items() if k.startswith(dataset_name)}
        if not dataset_dfs:
            continue

        print(f"\n{'='*60}")
        print(f"{dataset_name} Label Distribution")
        print(f"{'='*60}")

        for split in ['train', 'val', 'test']:
            key = f"{dataset_name}_{split}"
            if key not in dataset_dfs:
                continue

            df = dataset_dfs[key]
            print(f"\n{split.upper()} ({len(df)} images):")
            print("-" * 40)
            for col in label_cols:
                count = int(df[col].sum())
                pct = (count / len(df)) * 100 if len(df) > 0 else 0
                print(f"  {col}: {count:5d} ({pct:5.1f}%)")


def save_harmonized_data(harmonized_data: Dict[str, pd.DataFrame], output_dir: str = './harmonized_labels'):
    os.makedirs(output_dir, exist_ok=True)

    print("\n" + "="*60)
    print("SAVING HARMONIZED DATA")
    print("="*60)

    for key, df in harmonized_data.items():
        output_path = os.path.join(output_dir, f"{key}.csv")
        df.to_csv(output_path, index=False)
        print(f"✅ Saved {key:20s}: {len(df):5d} rows → {output_path}")

    print("\n📦 Creating combined files...")
    for split in ['train', 'val', 'test']:
        split_dfs = [v for k, v in harmonized_data.items() if k.endswith(f'_{split}')]
        if split_dfs:
            combined = pd.concat(split_dfs, ignore_index=True)
            output_path = os.path.join(output_dir, f"combined_{split}.csv")
            combined.to_csv(output_path, index=False)
            print(f"✅ Saved combined_{split:5s}: {len(combined):5d} rows → {output_path}")

    print(f"\n✨ All files saved to: {output_dir}")


def verify_images(harmonized_data: Dict[str, pd.DataFrame]):
    print("\n" + "="*60)
    print("VERIFYING IMAGE PATHS")
    print("="*60)

    all_good = True
    for key, df in sorted(harmonized_data.items()):
        missing = df[~df['image_path'].apply(os.path.exists)]
        if len(missing) > 0:
            print(f"❌ {key:20s}: {len(missing)} missing images")
            all_good = False
        else:
            print(f"✅ {key:20s}: All {len(df)} images found")

    if all_good:
        print("\n🎉 All image paths verified successfully!")
    else:
        print("\n⚠️  Some images are missing - check the paths above")


# =============================================================================
# CONFIG (LODO: Train=ODIR + RFMiD_v2, Test=RFMiD_v1)
# =============================================================================

SEED = 42
SAVE_DIR = './results_lodo_mixup/fold_test_RFMiD_v1'
TEST_DOMAIN = "RFMiD_v1"
TRAIN_DOMAINS = "ODIR + RFMiD_v2"
MIXUP_ALPHA = 0.2

TEST_CSV = '/kaggle/working/harmonized_labels/RFMiD_v1_test.csv'
TRAIN_CSVS = [
    '/kaggle/working/harmonized_labels/ODIR_train.csv',
    '/kaggle/working/harmonized_labels/RFMiD_v2_train.csv'
]
VAL_CSVS = [
    '/kaggle/working/harmonized_labels/ODIR_val.csv',
    '/kaggle/working/harmonized_labels/RFMiD_v2_val.csv'
]

# =============================================================================
# REPRODUCIBILITY
# =============================================================================

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ["PYTHONHASHSEED"] = str(seed)

# =============================================================================
# DATASET
# =============================================================================

class RetinalDataset(Dataset):
    def __init__(self, csv_path, domain_id, transform=None):
        self.data = pd.read_csv(csv_path)
        self.domain_id = domain_id
        self.transform = transform
        self.label_cols = ["N", "D", "G", "C", "A", "H", "M", "O"]

        dup = self.data.duplicated(subset=["image_path"], keep="first").sum()
        if dup > 0:
            print(f"[WARN] {dup} duplicates found, keeping first")
        self.data = self.data.drop_duplicates(subset=["image_path"]).reset_index(drop=True)

        valid_mask = self.data["image_path"].apply(os.path.exists)
        missing = (~valid_mask).sum()
        if missing > 0:
            print(f"[WARN] Dropping {missing} missing images")
        self.data = self.data.loc[valid_mask].reset_index(drop=True)
        print(f"[INFO] Domain {domain_id}: Loaded {len(self.data)} valid images")

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        try:
            image = Image.open(row["image_path"]).convert("RGB")
        except Exception:
            image = Image.new("RGB", (224, 224), color=(128, 128, 128))

        labels = torch.tensor(row[self.label_cols].values.astype("float32"))
        if self.transform:
            image = self.transform(image)

        return image, labels, self.domain_id

# =============================================================================
# POSITIVE CLASS WEIGHTS
# =============================================================================

def calculate_pos_weights(datasets, clip_min=0.5, clip_max=50.0):
    all_labels = []
    for ds in datasets:
        all_labels.append(ds.data[["N", "D", "G", "C", "A", "H", "M", "O"]].values)

    combined = np.vstack(all_labels)
    pos = combined.sum(axis=0)
    neg = len(combined) - pos
    raw = neg / (pos + 1e-5)
    clipped = np.clip(raw, clip_min, clip_max)

    print("\n[INFO] Positive class weights (from training domains):")
    for i, col in enumerate(["N", "D", "G", "C", "A", "H", "M", "O"]):
        print(f"  {col}: {clipped[i]:.2f}")

    return torch.tensor(clipped, dtype=torch.float32)

# =============================================================================
# MODEL (MobileNetV3-Large) - FIXED FEATURE DIM
# =============================================================================

class MixupMultiLabel(nn.Module):
    """
    MobileNetV3-Large backbone + custom head.
    IMPORTANT FIX:
      - If we set backbone.classifier = Identity, backbone(x) returns 960-d features (not 1280).
      - So in_features must be 960 (classifier[0].in_features), not classifier[-1].in_features.
    """
    def __init__(self, num_classes=8, dropout=0.3):
        super().__init__()

        # weights enum varies by torchvision version; keep it robust
        try:
            weights = models.MobileNet_V3_Large_Weights.IMAGENET1K_V2
        except Exception:
            weights = models.MobileNet_V3_Large_Weights.DEFAULT

        self.backbone = models.mobilenet_v3_large(weights=weights)

        # classifier[0] is Linear(960 -> 1280) in torchvision MobileNetV3-Large
        in_features = self.backbone.classifier[0].in_features  # <-- FIX (should be 960)

        # remove original classifier so forward returns 960-d features
        self.backbone.classifier = nn.Identity()

        self.classifier = nn.Sequential(
            nn.Dropout(dropout),
            nn.Linear(in_features, 512),
            nn.ReLU(),
            nn.BatchNorm1d(512),
            nn.Dropout(dropout),
            nn.Linear(512, num_classes),
        )

    def forward(self, x):
        feats = self.backbone(x)        # [B, 960]
        logits = self.classifier(feats) # [B, 8]
        return logits

# =============================================================================
# TRANSFORMS
# =============================================================================

def get_transforms(is_train=False):
    if is_train:
        return transforms.Compose([
            transforms.Resize((256, 256)),
            transforms.RandomCrop(224),
            transforms.RandomRotation(10),
            transforms.ColorJitter(brightness=0.2, contrast=0.2),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        ])
    else:
        return transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        ])

# =============================================================================
# METRICS
# =============================================================================

def compute_metrics(labels, probs, thresholds=None):
    n_classes = labels.shape[1]

    aucs = []
    for i in range(n_classes):
        if len(np.unique(labels[:, i])) > 1:
            aucs.append(roc_auc_score(labels[:, i], probs[:, i]))
        else:
            aucs.append(np.nan)

    aps = []
    for i in range(n_classes):
        if len(np.unique(labels[:, i])) > 1:
            aps.append(average_precision_score(labels[:, i], probs[:, i]))
        else:
            aps.append(np.nan)

    if thresholds is None:
        thresholds = np.full(n_classes, 0.5)

    preds = (probs >= thresholds).astype(int)
    f1 = f1_score(labels, preds, average="macro", zero_division=0)

    return {
        "mAUC": float(np.nanmean(aucs)),
        "mAP": float(np.nanmean(aps)),
        "per_class_auc": aucs,
        "per_class_ap": aps,
        "macro_f1": float(f1),
    }

def find_optimal_thresholds(labels, probs):
    n_classes = labels.shape[1]
    thresholds = []
    search_range = np.linspace(0.05, 0.95, 91)

    for i in range(n_classes):
        best_f1, best_t = 0.0, 0.5
        if len(np.unique(labels[:, i])) > 1:
            for t in search_range:
                preds = (probs[:, i] >= t).astype(int)
                f1 = f1_score(labels[:, i], preds, zero_division=0)
                if f1 > best_f1:
                    best_f1, best_t = f1, t
        thresholds.append(best_t)

    return np.array(thresholds)

# =============================================================================
# MIXUP TRAINING
# =============================================================================

def train_epoch_mixup(model, loaders_dict, criterion, optimizer, device, mixup_alpha=0.2):
    model.train()
    losses = []
    mixup_stats = {'total_batches': 0, 'mixup_batches': 0}

    domain_iters = {k: iter(v) for k, v in loaders_dict.items()}
    domain_ids = list(loaders_dict.keys())
    max_batches = max(len(loader) for loader in loaders_dict.values())

    pbar = tqdm(range(max_batches), desc="Train (Mixup)", leave=False)

    for _ in pbar:
        do_mixup = (len(domain_ids) >= 2) and (np.random.rand() > 0.5)

        if do_mixup:
            d1, d2 = np.random.choice(domain_ids, size=2, replace=False)

            try:
                imgs1, labels1, _ = next(domain_iters[d1])
            except StopIteration:
                domain_iters[d1] = iter(loaders_dict[d1])
                imgs1, labels1, _ = next(domain_iters[d1])

            try:
                imgs2, labels2, _ = next(domain_iters[d2])
            except StopIteration:
                domain_iters[d2] = iter(loaders_dict[d2])
                imgs2, labels2, _ = next(domain_iters[d2])

            min_size = min(imgs1.size(0), imgs2.size(0))
            imgs1, labels1 = imgs1[:min_size], labels1[:min_size]
            imgs2, labels2 = imgs2[:min_size], labels2[:min_size]

            lam = np.random.beta(mixup_alpha, mixup_alpha)

            mixed_imgs = lam * imgs1 + (1 - lam) * imgs2
            mixed_labels = lam * labels1 + (1 - lam) * labels2

            mixed_imgs = mixed_imgs.to(device)
            mixed_labels = mixed_labels.to(device)

            optimizer.zero_grad()
            logits = model(mixed_imgs)
            loss = criterion(logits, mixed_labels)

            mixup_stats['mixup_batches'] += 1

        else:
            d = int(np.random.choice(domain_ids))
            try:
                imgs, labels, _ = next(domain_iters[d])
            except StopIteration:
                domain_iters[d] = iter(loaders_dict[d])
                imgs, labels, _ = next(domain_iters[d])

            imgs, labels = imgs.to(device), labels.to(device)

            optimizer.zero_grad()
            logits = model(imgs)
            loss = criterion(logits, labels)

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()

        losses.append(loss.item())
        mixup_stats['total_batches'] += 1
        pbar.set_postfix({'loss': f"{loss.item():.4f}"})

    return {
        "loss": float(np.mean(losses)),
        "mixup_ratio": mixup_stats['mixup_batches'] / max(1, mixup_stats['total_batches'])
    }

@torch.no_grad()
def validate(model, loader, criterion, device, thresholds=None):
    model.eval()
    losses, all_probs, all_labels = [], [], []

    for batch in tqdm(loader, desc="Val", leave=False):
        if len(batch) == 3:
            images, labels, _ = batch
        else:
            images, labels = batch

        images, labels = images.to(device), labels.to(device)
        logits = model(images)
        loss = criterion(logits, labels)

        losses.append(loss.item())
        all_probs.append(torch.sigmoid(logits).cpu().numpy())
        all_labels.append(labels.cpu().numpy())

    probs = np.vstack(all_probs)
    labels = np.vstack(all_labels)
    metrics = compute_metrics(labels, probs, thresholds)
    metrics["loss"] = float(np.mean(losses))
    return metrics, probs, labels

# =============================================================================
# PLOTS
# =============================================================================

def plot_training_curves(history, save_dir):
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    epochs = range(1, len(history['train_loss']) + 1)

    axes[0].plot(epochs, history['train_loss'], 'b-', linewidth=2, label='Train Loss', marker='o')
    axes[0].plot(epochs, history['val_loss'], 'r-', linewidth=2, label='Val Loss', marker='s')
    axes[0].set_xlabel('Epoch', fontsize=12)
    axes[0].set_ylabel('Loss', fontsize=12)
    axes[0].set_title('Training and Validation Loss (Mixup)', fontsize=13, fontweight='bold')
    axes[0].legend(fontsize=11)
    axes[0].grid(True, alpha=0.3)

    axes[1].plot(epochs, history['val_auc'], 'r-', linewidth=2, label='Val mAUC', marker='s')
    axes[1].set_xlabel('Epoch', fontsize=12)
    axes[1].set_ylabel('mAUC', fontsize=12)
    axes[1].set_title('Validation mAUC (Mixup)', fontsize=13, fontweight='bold')
    axes[1].legend(fontsize=11)
    axes[1].grid(True, alpha=0.3)
    axes[1].set_ylim([0, 1.0])

    plt.tight_layout()
    plt.savefig(f'{save_dir}/training_curves.png', dpi=300, bbox_inches='tight')
    plt.close()
    print(f"[INFO] ✓ Saved training curves")

def plot_per_class_roc(labels, probs, test_domain, save_dir):
    class_names = ["N", "D", "G", "C", "A", "H", "M", "O"]
    n_classes = 8

    fig, axes = plt.subplots(2, 4, figsize=(20, 10))
    axes = axes.flatten()

    for i in range(n_classes):
        if len(np.unique(labels[:, i])) > 1:
            fpr, tpr, _ = roc_curve(labels[:, i], probs[:, i])
            auc = roc_auc_score(labels[:, i], probs[:, i])

            axes[i].plot(fpr, tpr, linewidth=2, label=f'AUC = {auc:.3f}')
            axes[i].plot([0, 1], [0, 1], 'k--', linewidth=1, alpha=0.5)
            axes[i].set_xlabel('False Positive Rate', fontsize=10)
            axes[i].set_ylabel('True Positive Rate', fontsize=10)
            axes[i].set_title(f'Class {class_names[i]}', fontsize=11, fontweight='bold')
            axes[i].legend(loc='lower right', fontsize=9)
            axes[i].grid(True, alpha=0.3)
        else:
            axes[i].text(0.5, 0.5, 'Single class\n(No ROC)', ha='center', va='center', fontsize=12)
            axes[i].set_title(f'Class {class_names[i]}', fontsize=11, fontweight='bold')

    plt.suptitle(f'Per-Class ROC Curves - Test on {test_domain} (Mixup)',
                 fontsize=14, fontweight='bold', y=0.995)
    plt.tight_layout()
    plt.savefig(f'{save_dir}/per_class_roc_curves.png', dpi=300, bbox_inches='tight')
    plt.close()
    print(f"[INFO] ✓ Saved per-class ROC curves")

def plot_macro_roc(labels, probs, test_domain, save_dir):
    n_classes = 8
    fig, ax = plt.subplots(figsize=(8, 6))

    all_fpr, all_tpr = [], []
    for i in range(n_classes):
        if len(np.unique(labels[:, i])) > 1:
            fpr, tpr, _ = roc_curve(labels[:, i], probs[:, i])
            all_fpr.append(fpr)
            all_tpr.append(tpr)

    mean_fpr = np.linspace(0, 1, 100)
    tprs = [np.interp(mean_fpr, fpr, tpr) for fpr, tpr in zip(all_fpr, all_tpr)]
    mean_tpr = np.mean(tprs, axis=0)
    macro_auc = np.trapz(mean_tpr, mean_fpr)

    ax.plot(mean_fpr, mean_tpr, linewidth=3,
            label=f'Macro-avg ROC (AUC = {macro_auc:.3f})', color='#2ecc71')
    ax.plot([0, 1], [0, 1], 'k--', linewidth=2, alpha=0.5, label='Random Classifier')
    ax.set_xlabel('False Positive Rate', fontsize=12)
    ax.set_ylabel('True Positive Rate', fontsize=12)
    ax.set_title(f'Macro-Average ROC Curve - Test on {test_domain} (Mixup)',
                 fontsize=13, fontweight='bold')
    ax.legend(loc='lower right', fontsize=11)
    ax.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig(f'{save_dir}/macro_roc_curve.png', dpi=300, bbox_inches='tight')
    plt.close()
    print(f"[INFO] ✓ Saved macro-average ROC curve")

def plot_per_class_metrics(test_metrics, save_dir):
    class_names = ["N", "D", "G", "C", "A", "H", "M", "O"]
    aucs = test_metrics['per_class_auc']
    aps = test_metrics['per_class_ap']

    aucs = [auc if not np.isnan(auc) else 0 for auc in aucs]
    aps = [ap if not np.isnan(ap) else 0 for ap in aps]

    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    x = np.arange(len(class_names))

    axes[0].bar(x, aucs, color='#3498db', alpha=0.8, edgecolor='black', linewidth=1.5)
    axes[0].set_xlabel('Disease Class', fontsize=11)
    axes[0].set_ylabel('AUC', fontsize=11)
    axes[0].set_title('Per-Class AUC (Mixup)', fontsize=12, fontweight='bold')
    axes[0].set_xticks(x)
    axes[0].set_xticklabels(class_names)
    axes[0].set_ylim([0, 1.0])
    axes[0].grid(True, alpha=0.3, axis='y')
    for i, auc in enumerate(aucs):
        axes[0].text(i, auc + 0.02, f'{auc:.2f}', ha='center', fontsize=9)

    axes[1].bar(x, aps, color='#2ecc71', alpha=0.8, edgecolor='black', linewidth=1.5)
    axes[1].set_xlabel('Disease Class', fontsize=11)
    axes[1].set_ylabel('Average Precision', fontsize=11)
    axes[1].set_title('Per-Class Average Precision (Mixup)', fontsize=12, fontweight='bold')
    axes[1].set_xticks(x)
    axes[1].set_xticklabels(class_names)
    axes[1].set_ylim([0, 1.0])
    axes[1].grid(True, alpha=0.3, axis='y')
    for i, ap in enumerate(aps):
        axes[1].text(i, ap + 0.02, f'{ap:.2f}', ha='center', fontsize=9)

    plt.tight_layout()
    plt.savefig(f'{save_dir}/per_class_metrics.png', dpi=300, bbox_inches='tight')
    plt.close()
    print(f"[INFO] ✓ Saved per-class metrics chart")

# =============================================================================
# MAIN
# =============================================================================

if __name__ == "__main__":
    # Optional: harmonize + save CSVs
    if RUN_HARMONIZATION:
        harmonized_data = harmonize_all_datasets()
        print_statistics(harmonized_data)
        verify_images(harmonized_data)
        save_harmonized_data(harmonized_data)

    set_seed(SEED)
    os.makedirs(SAVE_DIR, exist_ok=True)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    print("="*80)
    print(f"LODO FOLD (SWAPPED) with MIXUP DG: Test on {TEST_DOMAIN}")
    print(f"Training on: {TRAIN_DOMAINS}")
    print("="*80)
    print(f"Device: {device}")
    print(f"Seed: {SEED}")
    print(f"Mixup Alpha: {MIXUP_ALPHA}")
    print("="*80)

    # Load datasets
    print("\n[INFO] Loading datasets...")
    train_datasets = [RetinalDataset(csv, domain_id=i, transform=get_transforms(True))
                      for i, csv in enumerate(TRAIN_CSVS)]
    val_datasets = [RetinalDataset(csv, domain_id=i, transform=get_transforms(False))
                    for i, csv in enumerate(VAL_CSVS)]
    test_dataset = RetinalDataset(TEST_CSV, domain_id=999, transform=get_transforms(False))

    print(f"\n[INFO] Total train: {sum(len(ds) for ds in train_datasets)} images")
    print(f"[INFO] Total val: {sum(len(ds) for ds in val_datasets)} images")
    print(f"[INFO] Test: {len(test_dataset)} images")

    # Pos weights from training domains only
    pos_weights = calculate_pos_weights(train_datasets).to(device)

    # DataLoaders
    g = torch.Generator().manual_seed(SEED)
    train_loaders = {
        i: DataLoader(ds, batch_size=32, shuffle=True, num_workers=4,
                      pin_memory=True, generator=g)
        for i, ds in enumerate(train_datasets)
    }

    combined_val = ConcatDataset(val_datasets)
    val_loader = DataLoader(combined_val, batch_size=32, shuffle=False,
                            num_workers=4, pin_memory=True)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False,
                             num_workers=4, pin_memory=True)

    # Model
    print("\n[INFO] Initializing MobileNetV3-Large Mixup model (FIXED)...")
    model = MixupMultiLabel().to(device)

    criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weights)

    optimizer = optim.Adam([
        {'params': model.backbone.parameters(), 'lr': 1e-4},
        {'params': model.classifier.parameters(), 'lr': 1e-3}
    ], weight_decay=1e-4)

    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='max', factor=0.5, patience=3
    )

    history = {'train_loss': [], 'val_loss': [], 'val_auc': []}

    # Train
    best_val_auc, patience_counter = 0.0, 0
    print("\n[INFO] Training started with Mixup DG...")
    print("-"*80)

    for epoch in range(50):
        train_metrics = train_epoch_mixup(
            model, train_loaders, criterion, optimizer, device, mixup_alpha=MIXUP_ALPHA
        )
        val_metrics, _, _ = validate(model, val_loader, criterion, device)
        scheduler.step(val_metrics['mAUC'])

        history['train_loss'].append(train_metrics['loss'])
        history['val_loss'].append(val_metrics['loss'])
        history['val_auc'].append(val_metrics['mAUC'])

        print(f"Epoch {epoch+1:02d} | Train Loss: {train_metrics['loss']:.4f} "
              f"| Val mAUC: {val_metrics['mAUC']:.4f} | Mixup: {train_metrics['mixup_ratio']:.1%}")

        if val_metrics['mAUC'] > best_val_auc:
            best_val_auc = val_metrics['mAUC']
            torch.save(model.state_dict(), f'{SAVE_DIR}/best_model.pth')
            print(f"  ✓ Saved best model (val mAUC: {best_val_auc:.4f})")
            patience_counter = 0
        else:
            patience_counter += 1

        if patience_counter >= 10:
            print(f"\n[INFO] Early stopping at epoch {epoch+1}")
            break

    # Load best
    print(f"\n[INFO] Loading best model...")
    model.load_state_dict(torch.load(f'{SAVE_DIR}/best_model.pth', map_location=device))
    print(f"[INFO] Best validation mAUC: {best_val_auc:.4f}")

    # Thresholds on validation
    print(f"\n[INFO] Finding optimal thresholds on validation...")
    _, val_probs, val_labels = validate(model, val_loader, criterion, device)
    thresholds = find_optimal_thresholds(val_labels, val_probs)

    class_names = ["N", "D", "G", "C", "A", "H", "M", "O"]
    print("[INFO] Optimal thresholds:")
    for c, t in zip(class_names, thresholds):
        print(f"  {c}: {t:.3f}")

    # Test
    print(f"\n[INFO] Testing on {TEST_DOMAIN}...")
    print("-"*80)
    test_metrics, test_probs, test_labels = validate(
        model, test_loader, criterion, device, thresholds
    )

    print("\n" + "="*80)
    print(f"TEST RESULTS - {TEST_DOMAIN} (Mixup DG)")
    print("="*80)
    print(f"mAUC:      {test_metrics['mAUC']:.4f}")
    print(f"mAP:       {test_metrics['mAP']:.4f}")
    print(f"Macro F1:  {test_metrics['macro_f1']:.4f}")
    print("="*80)

    print(f"\n{'Class':<8} {'AUC':<10} {'AP':<10}")
    print("-"*30)
    for i, cls in enumerate(class_names):
        auc = test_metrics['per_class_auc'][i]
        ap = test_metrics['per_class_ap'][i]
        auc_str = f"{auc:.4f}" if not np.isnan(auc) else "N/A"
        ap_str = f"{ap:.4f}" if not np.isnan(ap) else "N/A"
        print(f"{cls:<8} {auc_str:<10} {ap_str:<10}")
    print("-"*30)

    # Save results
    print("\n[INFO] Saving results...")
    results_df = pd.DataFrame([{
        'method': 'Mixup',
        'backbone': 'mobilenet_v3_large',
        'test_domain': TEST_DOMAIN,
        'train_domains': TRAIN_DOMAINS,
        'mAUC': test_metrics['mAUC'],
        'mAP': test_metrics['mAP'],
        'macro_f1': test_metrics['macro_f1'],
        'best_val_auc': best_val_auc,
        'mixup_alpha': MIXUP_ALPHA
    }])
    results_df.to_csv(f'{SAVE_DIR}/test_results.csv', index=False)
    print(f"[INFO] ✓ Saved test_results.csv")

    # Visualizations
    print("\n[INFO] Generating visualizations...")
    plot_training_curves(history, SAVE_DIR)
    plot_per_class_roc(test_labels, test_probs, TEST_DOMAIN, SAVE_DIR)
    plot_macro_roc(test_labels, test_probs, TEST_DOMAIN, SAVE_DIR)
    plot_per_class_metrics(test_metrics, SAVE_DIR)

    print("\n" + "="*80)
    print(f"✓ MIXUP LODO FOLD (SWAPPED) COMPLETE! (MobileNetV3-Large FIXED)")
    print(f"✓ Results saved to: {SAVE_DIR}/")
    print("="*80)


HARMONIZING DATASETS

Processing train split...
RFMiD_v2 train: Found 507, Skipped 2

Processing val split...

Processing test split...
RFMiD_v2 test: Found 170, Skipped 4

DATASET STATISTICS

📊 Sample Counts:
----------------------------------------
ODIR_test           :  2000 images
ODIR_train          :  7000 images
ODIR_val            :  1000 images
RFMiD_v1_test       :   640 images
RFMiD_v1_train      :  1920 images
RFMiD_v1_val        :   640 images
RFMiD_v2_test       :   170 images
RFMiD_v2_train      :   507 images
RFMiD_v2_val        :   177 images
----------------------------------------
Total Train         :  9427 images
Total Val           :  1817 images
Total Test          :  2810 images
Grand Total         : 14054 images

ODIR Label Distribution

TRAIN (7000 images):
----------------------------------------
  N:  2280 ( 32.6%)
  D:  2256 ( 32.2%)
  G:   430 (  6.1%)
  C:   424 (  6.1%)
  A:   328 (  4.7%)
  H:   206 (  2.9%)
  M:   348 (  5.0%)
  O:  1958 ( 28.0%)

VAL

100%|██████████| 21.1M/21.1M [00:00<00:00, 123MB/s] 



[INFO] Training started with Mixup DG...
--------------------------------------------------------------------------------


Epoch 01 | Train Loss: 0.8981 | Val mAUC: 0.7770 | Mixup: 53.9%
  ✓ Saved best model (val mAUC: 0.7770)


Epoch 02 | Train Loss: 0.7058 | Val mAUC: 0.7722 | Mixup: 44.3%


...

Epoch 23 | Train Loss: 0.3716 | Val mAUC: 0.8113 | Mixup: 54.3%

[INFO] Early stopping at epoch 23

[INFO] Loading best model...
[INFO] Best validation mAUC: 0.8197

[INFO] Finding optimal thresholds on validation...


[INFO] Optimal thresholds:
  N: 0.640
  D: 0.220
  G: 0.460
  C: 0.840
  A: 0.690
  H: 0.290
  M: 0.790
  O: 0.180

[INFO] Testing on RFMiD_v1...
--------------------------------------------------------------------------------



TEST RESULTS - RFMiD_v1 (Mixup DG)
mAUC:      0.8384
mAP:       0.5037
Macro F1:  0.4053

Class    AUC        AP        
------------------------------
N        0.9296     0.7354    
D        0.8880     0.6530    
G        0.7009     0.3660    
C        0.8785     0.6806    
A        0.8695     0.3952    
H        0.8153     0.0084    
M        0.9748     0.6513    
O        0.6509     0.5392    
------------------------------

[INFO] Saving results...
[INFO] ✓ Saved test_results.csv

[INFO] Generating visualizations...
[INFO] ✓ Saved training curves
[INFO] ✓ Saved per-class ROC curves


/tmp/ipykernel_55/4249279455.py:705: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  macro_auc = np.trapz(mean_tpr, mean_fpr)


[INFO] ✓ Saved macro-average ROC curve
[INFO] ✓ Saved per-class metrics chart

✓ MIXUP LODO FOLD (SWAPPED) COMPLETE! (MobileNetV3-Large FIXED)
✓ Results saved to: ./results_lodo_mixup/fold_test_RFMiD_v1/
